This notebook will be used to clean out the data files we got from different websites into a single CSV file.  

In [6]:
import numpy as np
import pandas as pd

In [7]:
#By the end of the notebook, we will create a relatively large data set that includes all of our data.


#This will be used to decide the start and end date of our largest data set. 
years_bounds=[]
data_sets_names=[] #Names of the data sets we will use at the end
column_names=[] #Names of columns at the end
feature_description={} #Description of eavery feature we have in addition to units

### Cleaning Faculty data
In this section, we clean faculty data obtained from NCES and we also interpolated the gapped year in the data.

In [8]:

#Import the data
faculty_read=pd.read_csv("NCES_data/Faculty_series_cleaned_1970-2022.csv")
faculty_read.columns=['year','faculty']
#Check data and the number of NAN entries. 
# faculty_read.info()
# print()
# print()

#If NAN>0, clean these rows
faculty_read=faculty_read.dropna()

#Check the format of the entires of each column
# print(faculty_read.head())
# print()

#Clean columns
def clean_column(text:str) -> int:
    # text = text[1:] # removes leading $
    return int(text.replace(',','')) # removes comma and converts to int

faculty_read['year']=faculty_read['year'].astype(int)
faculty_read['faculty']=faculty_read['faculty'].apply(clean_column)
# faculty_read.info()
# print()
# print()

#Find the start and end years to be used as keys when analyzing the data 
faculty_start, faculty_end = faculty_read['year'].iloc[0],faculty_read['year'].iloc[-1]
years_bounds.append(faculty_start)
years_bounds.append(faculty_end)

faculty_data=faculty_read

#cheack that there are no missing years (gaps)
print(faculty_data.head())
print()
print(f"The year range is {(faculty_end-faculty_start+1)}")
print(f"The number of missing years is {(faculty_end-faculty_start+1)- len(faculty_data)}")

data_sets_names.append('faculty_data')
column_names.append('faculty')
feature_description['faculty']=["# of faculty in a given year","faculty"]

   year  faculty
0  1970   474000
1  1971   492000
2  1972   500000
3  1973   527000
4  1974   567000

The year range is 53
The number of missing years is 15


Since there's a gap in the faculty data, we will have to find a way to work around this. One of the easiest ways is to use an auto average fill. This is basically filling evey missing year by the mean of the number of faculty of the pervious and the next year. 

In [4]:
years_old=faculty_data['year'].to_numpy()
# print(years_old)
years_new=np.arange(faculty_start,faculty_end+1,1)
# print(years_new)
# print(len(years_new)-len(years_old))
faculty_data_interpolated=np.zeros((len(years_new),2)) #array of zeros to store the data

iter=0
for year in years_new:
             faculty_data_interpolated[iter][0]=year
             if year in years_old: faculty_data_interpolated[iter][1]=faculty_data[faculty_data['year']==year]['faculty'].sum()
             else: faculty_data_interpolated[iter][1]=faculty_data[(faculty_data['year']==year-1) | (faculty_data['year']==year+1)]['faculty'].mean()
             iter+=1

faculty_data_interpolated=pd.DataFrame(faculty_data_interpolated)
faculty_data_interpolated.columns=['year','faculty']

print(faculty_data_interpolated.head(10))
print()
print(faculty_data.head(10))


     year   faculty
0  1970.0  474000.0
1  1971.0  492000.0
2  1972.0  500000.0
3  1973.0  527000.0
4  1974.0  567000.0
5  1975.0  628000.0
6  1976.0  633000.0
7  1977.0  678000.0
8  1978.0  676500.0
9  1979.0  675000.0

   year  faculty
0  1970   474000
1  1971   492000
2  1972   500000
3  1973   527000
4  1974   567000
5  1975   628000
6  1976   633000
7  1977   678000
8  1979   675000
9  1980   686000


### Cleaning non-FRED Economic Factors
In this section, we clean the economic data not obtained from FRED. Each set of these data has special format so we treat each of them seperately. We deal with 3 different sets of data

1. NSF Awards

In [9]:
#Import the data
NSF_read=pd.read_csv("Economic_factors/NSF_awards_2011-2023.csv")
#Check data and the number of NAN entries. 
# NSF_read.info()
# print()
# print()

#If NAN>0, clean these rows
# faculty_data=faculty_data.dropna()

#Check the format of the entires of each column
# print(NSF_read.head())
# print()

#Clean columns
def clean_column(text:str) -> float:
    text = text[1:-1] # removes leading $ and tailing M
    return float(text) #returns a float

NSF_read["Award Obligation to Date "]=NSF_read["Award Obligation to Date "].apply(clean_column)
# NSF_read.info()
# print()
# print()

#All this data is split into years so we are gonna create a new data frame where every row corresponds to a year
years=(NSF_read['Fiscal Year'].unique()) #finds unique years
years.sort() #sorts the array

NSF_data=np.zeros((len(years),2)) #array of zeros to store the data
iter=0
for year in years:
             NSF_data[iter][0],NSF_data[iter][1]=year, NSF_read[NSF_read['Fiscal Year']==year]["Award Obligation to Date "].sum()
             iter+=1
NSF_data=pd.DataFrame(NSF_data)
NSF_data.columns=['year','NSF_awards']

#Find the start and end years to be used as keys when analyzing the data 
NSF_start, NSF_end =years[0], years[-1]
years_bounds.append(NSF_start)
years_bounds.append(NSF_end)

#cheack that there are no missing years (gaps)
print(NSF_data.head())
print()
print(f"The year range is {(NSF_end-NSF_start+1)}")
print(f"The number of missing years is {(NSF_end-NSF_start+1)- len(NSF_data)}")

data_sets_names.append('NSF_data')
column_names.append('NSF_awards')

feature_description['NSF_awards']=["Total amount of NSF awards in a given year","Million $"]


     year  NSF_awards
0  2011.0     5460.18
1  2012.0     5542.50
2  2013.0     5361.64
3  2014.0     6159.59
4  2015.0     5678.03

The year range is 13
The number of missing years is 0


2. Inflation rate given as a percent

In [10]:
#Import the data
inflation_percent_data=pd.read_csv("Economic_factors/inflation_rate_annual_percent_1980_2024_annual.csv")
inflation_percent_data.columns=['year','inflation_rate']

#Check data and the number of NAN entries. 
# inflation_percent_data.info()
# print()
# print()


#If NAN>0, clean these rows
#inflation_percent_data=inflation_percent_data.dropna()

#Check the format of the entires of each column
# print(inflation_percent_data.head())
# print()

#Drops previous predictions
inflation_percent_data=inflation_percent_data[inflation_percent_data['year']<2025]

# Find the start and end years to be used as keys when analyzing the data 
inflation_percent_start, inflation_percent_end =inflation_percent_data['year'][0], inflation_percent_data['year'][len(inflation_percent_data)-1]
years_bounds.append(inflation_percent_start)
years_bounds.append(inflation_percent_end)

# #cheack that there are no missing years (gaps)
print(inflation_percent_data.head())
print()
print(f"The year range is {(inflation_percent_end-inflation_percent_start+1)}")
print(f"The number of missing years is {(inflation_percent_end-inflation_percent_start+1)- len(inflation_percent_data)}")

data_sets_names.append('inflation_percent_data')
column_names.append('inflation_rate')


feature_description['inflation_rate']=["Inflation rate, average consumer prices, annual percent change","%"]

   year  inflation_rate
0  1980            13.5
1  1981            10.4
2  1982             6.2
3  1983             3.2
4  1984             4.4

The year range is 45
The number of missing years is 0


3. The fedrel budget for higher educations

In [11]:

#Import the data
Fed_Budget_Read = pd.read_csv("Economic_factors/Federal_budget_education_1980_2021_annual.csv")
Fed_Budget_Read.columns=['year','budget']
Fed_Budget_Read['President']=Fed_Budget_Read['year']
#Fed_Budget_Read.info()

#If NAN>0, clean these rows
Fed_Budget_Read=Fed_Budget_Read.dropna()


def clean_column(text:str):
    text = text[0:4] # pickes the year part
    return text

Fed_Budget_Read['year']=Fed_Budget_Read['year'].apply(clean_column)

def clean_column(text:str) -> int:
    if text[4:5]=="\n": return text[6:7] # pickes President or Appropriation
    elif text[5:6]=="\n": return text[6:7]
    else: return text[5:6]
    
Fed_Budget_Read['President']=Fed_Budget_Read['President'].apply(clean_column)


indices_drop=Fed_Budget_Read[Fed_Budget_Read['year']=="Majo"].index #Finds the indices where it's not a year
Fed_Budget_Read.drop(indices_drop, inplace=True) #Deletes those rows
Fed_Budget_Read.index=np.arange(len(Fed_Budget_Read)) #Renames indices in order 


Fed_Budget_Read['year']=Fed_Budget_Read['year'].astype(int)
Fed_Budget_Read['budget']=(Fed_Budget_Read['budget'].astype(float))/1000 #to be given in millions $
Fed_Budget_Read['PA_budget_diff']=Fed_Budget_Read['budget'].diff()

# print(Fed_Budget_Read.sample(20))
#print()


# All this data is split into years so we are gonna create a new data frame where every row corresponds to a year
years=(Fed_Budget_Read['year'].unique()) #finds unique years
years.sort() #sorts the array


Fed_Budget_data=np.zeros((len(years),3)) #array of zeros to store the data
iter=0
for year in years:
             Fed_Budget_data[iter][0]=year
             Fed_Budget_data[iter][1]=Fed_Budget_Read[(Fed_Budget_Read['year']==year) & (Fed_Budget_Read['President']=="A")]['budget'].sum()
             Fed_Budget_data[iter][2]=Fed_Budget_Read[(Fed_Budget_Read['year']==year) & (Fed_Budget_Read['President']=="A")]['PA_budget_diff'].sum()
             iter+=1
Fed_Budget_data=pd.DataFrame(Fed_Budget_data)
Fed_Budget_data.columns=['year','Fed_Budget', 'PA_Budget_diff']



# Find the start and end years to be used as keys when analyzing the data 
Fed_Budget_start, Fed_Budget_end =Fed_Budget_data['year'][0], Fed_Budget_data['year'].iloc[len(Fed_Budget_data)-1]
years_bounds.append(Fed_Budget_start)
years_bounds.append(Fed_Budget_end)

#cheack that there are no missing years (gaps)
print(Fed_Budget_data.head())
print()
print(f"The year range is {(Fed_Budget_end-Fed_Budget_start+1)}")
print(f"The number of missing years is {(Fed_Budget_end-Fed_Budget_start+1)- len(Fed_Budget_data)}")

data_sets_names.append('Fed_Budget_data')
data_sets_names.append('Fed_Budget_data')
column_names.append('Fed_Budget')
column_names.append( 'PA_Budget_diff')

feature_description['Fed_Budget']=["Federal budget for postsecondary education in a given year","Million $"]
feature_description['PA_Budget_diff']=["The difference between the president's proposed budget and the actual fedral budget in a given year ","Million $"]


     year  Fed_Budget  PA_Budget_diff
0  1980.0    5645.806           0.000
1  1981.0    6934.329        -200.981
2  1982.0    7282.939         412.064
3  1983.0    7358.132        1969.827
4  1984.0    6891.971         671.433

The year range is 42.0
The number of missing years is 0.0


### Cleaning the FRED Economic Data

We start by defining a function that will clean any data set obtained from FRED

In [13]:
def Clean_FRED_Set( data_set_read, col1_name, col2_name):
    '''
    Takes a downloaded FRED CSV file and puts it into a usable format for our time series purposes.

    Inputs
    data_set_read: That's the data set that is being read from the CSV file as a pandas data frame.
    col1_name:  a string represents the desired name of the first column.
    col2_name:  a string represents the desired name of the second column.
    
    Outputs:
    (data,start_year, end year)
    data: a pandas data frame with the formats cleaned
    start_year: the year when the data starts
    end_year: the year when the data ends

    '''

    #Set the name of the columns
    col1,col2=data_set_read.columns
    #If NAN>0, clean these rows
    data_set_read=data_set_read.dropna()

    # #Check the format of the entires of each column

    #Clean columns
    def clean_column(text:str) -> float:
        text = text[0:4] # removes month and day
        return int(text) #returns an int

    data_set_read[col1]=data_set_read[col1].apply(clean_column)
   

    # All this data is split into years so we are gonna create a new data frame where every row corresponds to a year
    years=(data_set_read[col1].unique()) #finds unique years
    years.sort() #sorts the array

    data=np.zeros((len(years),2)) #array of zeros to store the data
    iter=0
    for year in years:
                data[iter][0],data[iter][1]=year, data_set_read[data_set_read[col1]==year][col2].mean()
                iter+=1

    data=pd.DataFrame(data)
    data.columns=[col1_name,col2_name]

    #Find the start and end years to be used as keys when analyzing the data 
    start_year, end_year =years[0], years[-1]


    return([data,start_year,end_year])


Apply our cleaning function to the Nominal GDP

In [14]:
#Import the data
Nominal_GDP_read=pd.read_csv("Economic_factors/FRED_data/Nominal_GDP_USA.csv")

GDP_data, GDP_start, GDP_end = Clean_FRED_Set(Nominal_GDP_read, 'year', 'GDP')

years_bounds.append(GDP_start)
years_bounds.append(GDP_end)


#cheack that there are no missing years (gaps)
print(GDP_data.head())
print()
print(f"The year range is {(GDP_end-GDP_start+1)}")
print(f"The number of missing years is {(GDP_end - GDP_start+1)- len(GDP_data)}")

data_sets_names.append('GDP_data')
column_names.append('GDP')
feature_description['GDP']=["Nominal GDP","Million $"]



     year           GDP
0  1950.0  74956.847656
1  1951.0  86728.324219
2  1952.0  91835.199219
3  1953.0  97304.398438
4  1954.0  97637.273438

The year range is 75
The number of missing years is 0


Apply our cleaning function to CPI inflation rate

In [ ]:
#Import the data
inflation_CPI_read=pd.read_csv("Economic_factors/FRED_data/CPI_inflation_rate_measure_1913_2024_monthly.csv")

CPI_data, CPI_start, CPI_end = Clean_FRED_Set(inflation_CPI_read, 'year', 'CPI_inflation')

years_bounds.append(CPI_start)
years_bounds.append(CPI_end)


#cheack that there are no missing years (gaps)
print(CPI_data.head())
print()
print(f"The year range is {(CPI_end-CPI_start+1)}")
print(f"The number of missing years is {(CPI_end - CPI_start+1)- len(CPI_data)}")


data_sets_names.append('CPI_data')
column_names.append('CPI_inflation')
feature_description['CPI_inflation']=["Consumer Price Index for All Urban Consumers: Purchasing Power of the Consumer Dollar in U.S. City Average","1"]



     year  CPI_inflation
0  1913.0       9.883333
1  1914.0      10.016667
2  1915.0      10.108333
3  1916.0      10.883333
4  1917.0      12.825000

The year range is 112
The number of missing years is 0


Apply our cleaning function to the labor force by Bachelor Degree

In [15]:
#Import the data
Labor_force_BS_read=pd.read_csv("Economic_factors/FRED_data/Labor_force_BS_Degree_1992_2024_monthly.csv")

Labor_BS_data, Labor_BS_start, Labor_BS_end = Clean_FRED_Set(Labor_force_BS_read, 'year', 'Labor_BS')

years_bounds.append(Labor_BS_start)
years_bounds.append(Labor_BS_end)


#cheack that there are no missing years (gaps)
print(Labor_BS_data.head())
print()
print(f"The year range is {(Labor_BS_end-Labor_BS_start+1)}")
print(f"The number of missing years is {(Labor_BS_end - Labor_BS_start+1)- len(Labor_BS_data)}")


data_sets_names.append('Labor_BS_data')
column_names.append('Labor_BS')
feature_description['Labor_BS']=["Labor Force Participation Rate - Bachelor's Degree and Higher, 25 Yrs. & over","%"]



     year   Labor_BS
0  1992.0  81.341667
1  1993.0  81.133333
2  1994.0  81.141667
3  1995.0  80.966667
4  1996.0  80.508333

The year range is 33
The number of missing years is 0


Apply our cleaning function to Labor market conditions activity level

In [16]:
#Import the data
Labor_cond_read=pd.read_csv("Economic_factors/FRED_data/Labor_market_conditions_activity_level_1992_2024_monthly.csv")

Labor_cond_data, Labor_cond_start, Labor_cond_end = Clean_FRED_Set(Labor_cond_read, 'year', 'Labor_cond')

years_bounds.append(Labor_cond_start)
years_bounds.append(Labor_cond_end)


#cheack that there are no missing years (gaps)
print(Labor_cond_data.head())
print()
print(f"The year range is {(Labor_cond_end-Labor_cond_start+1)}")
print(f"The number of missing years is {(Labor_cond_end - Labor_cond_start+1)- len(Labor_cond_data)}")


data_sets_names.append('Labor_cond_data')
column_names.append('Labor_cond')

feature_description['Labor_cond']=["KC Fed Labor Market Conditions Index, Level of Activity Indicator ","1"]


     year  Labor_cond
0  1992.0   -0.807656
1  1993.0   -0.588828
2  1994.0    0.019888
3  1995.0    0.336564
4  1996.0    0.408764

The year range is 33
The number of missing years is 0


Apply our cleaning function to unemployment for Bachelor Degree

In [ ]:
#Import the data
Unemploy_BS_read=pd.read_csv("Economic_factors/FRED_data/Unemployment_rate_BS_Degree_1992_2024_monthly.csv")

Unemploy_BS_data, Unemploy_BS_start, Unemploy_BS_end = Clean_FRED_Set(Unemploy_BS_read, 'year', 'Unemploy_BS')

years_bounds.append(Unemploy_BS_start)
years_bounds.append(Unemploy_BS_end)


#cheack that there are no missing years (gaps)
print(Unemploy_BS_data.head())
print()
print(f"The year range is {(Unemploy_BS_end-Unemploy_BS_start+1)}")
print(f"The number of missing years is {(Unemploy_BS_end - Unemploy_BS_start+1)- len(Unemploy_BS_data)}")


data_sets_names.append('Unemploy_BS_data')
column_names.append('Unemploy_BS')
feature_description['Unemploy_BS']=["Unemployment Rate - Bachelor's Degree and Higher, 25 Yrs. & over","%"]



     year  Unemploy_BS
0  1992.0     3.158333
1  1993.0     2.941667
2  1994.0     2.583333
3  1995.0     2.450000
4  1996.0     2.258333

The year range is 33
The number of missing years is 0


Apply our cleaning function to unemployment rate

In [18]:
#Import the data
Unemploy_read=pd.read_csv("Economic_factors/FRED_data/Unemployment_rate_1948_2024_monthly.csv")

Unemploy_data, Unemploy_start, Unemploy_end = Clean_FRED_Set(Unemploy_read, 'year', 'Unemploy')

years_bounds.append(Unemploy_start)
years_bounds.append(Unemploy_end)


#cheack that there are no missing years (gaps)
print(Unemploy_data.head())
print()
print(f"The year range is {(Unemploy_end-Unemploy_start+1)}")
print(f"The number of missing years is {(Unemploy_end - Unemploy_start+1)- len(Unemploy_data)}")

data_sets_names.append('Unemploy_data')
column_names.append('Unemploy')
feature_description['Unemploy']=["Unemployment Rate","%"]


     year  Unemploy
0  1948.0  3.766667
1  1949.0  5.908333
2  1950.0  5.325000
3  1951.0  3.333333
4  1952.0  3.033333

The year range is 77
The number of missing years is 0


### Combining Data
In this section, we combine all the data into two CSV files: one with original faculty data (gapped) and one with interpolated faculty data.


Original faculty data (gapped)

In [15]:
# print(column_names)
# print(data_sets_names)
# print()

#globals()[data_sets_names[1]][[column_names[0],column_names[2]]]

first_year=np.min(years_bounds)
last_year=np.max(years_bounds)
all_years=np.arange(first_year,last_year+1,1)
data=np.zeros((len(all_years),len(column_names)+1))


year_iter=0
for year in all_years:
    col_iter=0
    data[year_iter][col_iter]=year 
    for col_name in column_names:
        data[year_iter][col_iter+1]=globals()[data_sets_names[col_iter]][globals()[data_sets_names[col_iter]]['year']==year][col_name].mean()
        col_iter+=1
    year_iter+=1 


#We now convert the 2d array into a data frame and assign names
new_column_names=np.concatenate((['year'],column_names))

data=pd.DataFrame(data)
data.columns=new_column_names

data.info()

data.to_csv('data_gapped.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            112 non-null    float64
 1   faculty         38 non-null     float64
 2   NSF_awards      13 non-null     float64
 3   inflation_rate  45 non-null     float64
 4   Fed_Budget      42 non-null     float64
 5   PA_Budget_diff  42 non-null     float64
 6   GDP             75 non-null     float64
 7   CPI_inflation   112 non-null    float64
 8   Labor_BS        33 non-null     float64
 9   Labor_cond      33 non-null     float64
 10  Unemploy_BS     33 non-null     float64
 11  Unemploy        77 non-null     float64
dtypes: float64(12)
memory usage: 10.6 KB


The interpolated faculty data

In [16]:
# print(column_names)
# print(data_sets_names)
# print()

#globals()[data_sets_names[1]][[column_names[0],column_names[2]]]

first_year=np.min(years_bounds)
last_year=np.max(years_bounds)
all_years=np.arange(first_year,last_year+1,1)
data_interpolated=np.zeros((len(all_years),len(column_names)+1))

data_sets_names[0]='faculty_data_interpolated'

year_iter=0
for year in all_years:
    col_iter=0
    data_interpolated[year_iter][col_iter]=year 
    for col_name in column_names:
        data_interpolated[year_iter][col_iter+1]=globals()[data_sets_names[col_iter]][globals()[data_sets_names[col_iter]]['year']==year][col_name].mean()
        col_iter+=1
    year_iter+=1 


#We now convert the 2d array into a data frame and assign names
new_column_names=np.concatenate((['year'],column_names))

data_interpolated=pd.DataFrame(data_interpolated)
data_interpolated.columns=new_column_names

data_interpolated.info()


data_interpolated.to_csv('data_interpolated.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            112 non-null    float64
 1   faculty         53 non-null     float64
 2   NSF_awards      13 non-null     float64
 3   inflation_rate  45 non-null     float64
 4   Fed_Budget      42 non-null     float64
 5   PA_Budget_diff  42 non-null     float64
 6   GDP             75 non-null     float64
 7   CPI_inflation   112 non-null    float64
 8   Labor_BS        33 non-null     float64
 9   Labor_cond      33 non-null     float64
 10  Unemploy_BS     33 non-null     float64
 11  Unemploy        77 non-null     float64
dtypes: float64(12)
memory usage: 10.6 KB


The data frame that describes each feature and states the units

In [22]:
feature_description=pd.DataFrame(feature_description)
feature_description.to_csv('data_description_and_units.csv', index=False)
feature_description

,faculty,NSF_awards,inflation_rate,Fed_Budget,PA_Budget_diff,GDP,Labor_BS,Labor_cond,Unemploy_BS,Unemploy
0,# of faculty in a given year,Total amount of NSF awards in a given year,"Inflation rate, average consumer prices, annua...",Federal budget for postsecondary education in ...,The difference between the president's propose...,Nominal GDP,Labor Force Participation Rate - Bachelor's De...,"KC Fed Labor Market Conditions Index, Level of...",Unemployment Rate - Bachelor's Degree and High...,Unemployment Rate
1,faculty,Million $,%,Million $,Million $,Million $,%,1,%,%
